In [14]:
import os
from datetime import datetime
from pathlib import Path
from sys import stdout
from openmm import unit, LangevinIntegrator
from openmm.app import HBonds, NoCutoff, PDBFile, Modeller, ForceField, Simulation, StateDataReporter, DCDReporter, PDBReporter
import urllib.request  
import numpy as np
from scipy.spatial.distance import squareform
import scipy.stats as stats
import matplotlib.pyplot as plt
import pandas as pd
from scipy.interpolate import griddata
import seaborn as sns
from matplotlib import colors
from IPython.display import set_matplotlib_formats


In [12]:

# Define the paths for input and output folders
input_folder = r'E:\MolecularDynamicsApp\src\MolecularDynamics\data\input'
output_folder = r'E:\MolecularDynamicsApp\src\MolecularDynamics\data\output'

# Automatically find the PDB file in the input folder
input_pdb_filename = None
for file in os.listdir(input_folder):
    if file.endswith(".pdb"):
        input_pdb_filename = file
        break

if input_pdb_filename is None:
    raise FileNotFoundError("No PDB file found in the input folder")

# Construct the full input path
input_pdb_path = os.path.join(input_folder, input_pdb_filename)

# Extract the base name from the input filename (without extension)
base_name = os.path.splitext(input_pdb_filename)[0]

# Generate a timestamp for unique output filenames
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Define the output filenames using the base name and the timestamp
output_init_with_hydrogens_filename = f'{base_name}_with_hydrogens_{timestamp}.pdb'
output_pdb_filename = f'{base_name}_out_{timestamp}.pdb'
output_dcd_filename = f'{base_name}_traj_{timestamp}.dcd'
output_csv_filename = f'{base_name}_{timestamp}.csv'

# Construct the full output paths
output_init_with_hydrogens_path = os.path.join(output_folder, output_init_with_hydrogens_filename)
output_pdb_path = os.path.join(output_folder, output_pdb_filename)
output_dcd_path = os.path.join(output_folder, output_dcd_filename)
output_csv_path = os.path.join(output_folder, output_csv_filename)

# Load the input PDB file
pdb = PDBFile(input_pdb_path)
print(pdb.topology)

# Create Modeller instance and load positions
modeller = Modeller(pdb.topology, pdb.positions)

# Load force field
forcefield = ForceField('amber14-all.xml')

# Add hydrogens
modeller.addHydrogens(forcefield)

# Write modified PDB file with hydrogens added
with open(output_init_with_hydrogens_path, 'w') as outfile:
    PDBFile.writeFile(modeller.topology, modeller.positions, outfile)

print(f"Modified PDB file with hydrogens saved to: {output_init_with_hydrogens_path}")

# Create system
system = forcefield.createSystem(modeller.topology, nonbondedMethod=app.NoCutoff, 
                                 nonbondedCutoff=3*nanometer, constraints=app.HBonds)

# Create integrator
integrator = LangevinIntegrator(303.15*kelvin, 1/picosecond, 2*femtoseconds)

# Create simulation
simulation = Simulation(modeller.topology, system, integrator)
simulation.context.setPositions(modeller.positions)

# Add reporters for state data and trajectory
simulation.reporters.append(StateDataReporter(sys.stdout, 1000, step=True, potentialEnergy=True, temperature=True))

# Minimize energy
simulation.minimizeEnergy(maxIterations=1000)
state1 = simulation.context.getState(getEnergy=True)
print(state1.getPotentialEnergy())

# Clear previous reporters
simulation.reporters = []

# Add new reporters for trajectory, PDB, and state data
simulation.reporters.append(DCDReporter(output_dcd_path, 1000))
simulation.reporters.append(PDBReporter(output_pdb_path, 1000))
simulation.reporters.append(StateDataReporter(sys.stdout, 1000, step=True, temperature=True, elapsedTime=True))
simulation.reporters.append(StateDataReporter(output_csv_path, 1000, step=True, time=True,
                                              potentialEnergy=True, totalEnergy=True, temperature=True))

# Run the simulation
simulation.step(5000)

# Clean up
del simulation

print(f"Output PDB file saved to: {output_pdb_path}")
print(f"Output DCD file saved to: {output_dcd_path}")
print(f"Output CSV file saved to: {output_csv_path}")


<Topology; 2 chains, 3 residues, 56 atoms, 61 bonds>
Modified PDB file with hydrogens saved to: E:\MolecularDynamicsApp\src\MolecularDynamics\data\output\A53c_mdr_with_hydrogens_20240624_181742.pdb
-1169.90478515625 kJ/mol
#"Step","Temperature (K)","Elapsed Time (s)"
1000,247.71561592279014,0.0009999275207519531
2000,259.52448266666767,0.6419177055358887
3000,305.4200221522492,1.2641496658325195
4000,339.2173254504953,1.8488011360168457
5000,343.9560302541323,2.4904043674468994
Output PDB file saved to: E:\MolecularDynamicsApp\src\MolecularDynamics\data\output\A53c_mdr_out_20240624_181742.pdb
Output DCD file saved to: E:\MolecularDynamicsApp\src\MolecularDynamics\data\output\A53c_mdr_traj_20240624_181742.dcd
Output CSV file saved to: E:\MolecularDynamicsApp\src\MolecularDynamics\data\output\A53c_mdr_20240624_181742.csv
